In [1]:
from constraint import *
from itertools import *

In [83]:
# Configuration and Setup
#
# Per day naming constants
M = 'monday'
T = 'tuesday'
W = 'wednesday'
TH = 'thursday'
F = 'friday'

In [84]:
# Configure number of free hours each day
#
# Sample :
#   free_hours_per_day[W] = 3  # Sets wednesday to 3 free hours

free_hours_per_day = {}
free_hours_per_day[M]  = 2  # Monday
free_hours_per_day[T]  = 1  # Tuesday
free_hours_per_day[W]  = 1  # Wednesday
free_hours_per_day[TH] = 1  # Thursday
free_hours_per_day[F]  = 1  # Friday

In [85]:
# Configure tasks to complete this week
#
# Sample tasks
#   tasks['name'] = { 'effort': 1 }
#   tasks['name'] = { 'effort': 3, 'due': M }

tasks = {}
tasks['laundry'] = { 'effort': 1, 'due': M }
tasks['pay bills'] = { 'effort': 1, 'due': T }
tasks['wash the car'] = { 'effort': 1 }

In [86]:
total_free_hours = 0
hour_to_day = {}
for day in [M,T,W,TH,F]:
    for hour in range(0,free_hours_per_day[day]):
      hour_to_day[total_free_hours] = day
      total_free_hours += 1

In [87]:
# hour_to_day is a mapping of hour in week to a day
hour_to_day

{0: 'monday',
 1: 'monday',
 2: 'tuesday',
 3: 'wednesday',
 4: 'thursday',
 5: 'friday'}

In [88]:
# total_free_hours is a sum of all the free hours this week 
total_free_hours

6

In [89]:
hours_in_week = range(0,total_free_hours)
hours_in_week

[0, 1, 2, 3, 4, 5]

In [90]:
#task_schedules = list(itertools.product(hours_in_week, repeat=len(tasks)))

In [91]:
# Begin defining the constraint solver

In [92]:
problem = Problem()
problem.addVariables(tasks.keys(),hours_in_week)

In [93]:
# Add a constraint so that no task begin on an identical hour
problem.addConstraint(AllDifferentConstraint())

In [94]:
# Get a solution
solution = problem.getSolution()
solution

{'laundry': 5, 'pay bills': 4, 'wash the car': 3}